# Breakdown data querying

In [ ]:
# From minicuber class

from pyproj import Transformer
from pyproj.aoi import AreaOfInterest
from pyproj.database import query_utm_crs_info

def bbox(lon_lat, xy_shape, resolution):

    utm_epsg = int(query_utm_crs_info(
        datum_name="WGS 84",
        area_of_interest=AreaOfInterest(lon_lat[0], lon_lat[1], lon_lat[0], lon_lat[1])
    )[0].code)

    transformer = Transformer.from_crs(4326, utm_epsg, always_xy=True)

    x_center, y_center = transformer.transform(*lon_lat)

    nx, ny = xy_shape
    
    x_left, x_right = x_center - resolution * (nx//2), x_center + resolution * (nx//2)

    y_top, y_bottom = y_center + resolution * (ny//2), y_center - resolution * (ny//2)

    return transformer.transform_bounds(x_left, y_bottom, x_right, y_top, direction = 'INVERSE') # left, bottom, right, top

def padded_bbox(bbox, xy_shape):
    left, bottom, right, top = bbox
    lat_extra = (top - bottom) / xy_shape[0] * 6
    lon_extra = (right - left) / xy_shape[1] * 6
    return left - lon_extra, bottom - lat_extra, right + lon_extra, top + lat_extra

In [ ]:
# From sentinel2.py

import os
import pystac_client
from pystac_client.stac_api_io import StacApiIO

URL = 'https://planetarycomputer.microsoft.com/api/stac/v1'

stac_api_io = StacApiIO()
stac_api_io.session.verify = os.environ.get('CURL_CA_BUNDLE')
catalog = pystac_client.Client.open(URL,stac_io=stac_api_io)

In [ ]:
# From minicuber.load_data and sentinel2

lon_lat = (7.7256, 46.9206) #lon-lat
xy_shape = (128, 128)
resolution = 10
time_interval = "2022-01-01/2022-01-31"

bbox_query = bbox(lon_lat=lon_lat, xy_shape=xy_shape, resolution=resolution)
padded_bbox_query = padded_bbox(bbox_query, xy_shape)

search = catalog.search(
        bbox = padded_bbox_query,
        collections=["sentinel-2-l2a"],
        datetime=time_interval
) # returns a ItermSearch object (search object of STAC API)

In [ ]:
# From sentinel2

import planetary_computer as pc 
import stackstac
import rasterio
from rasterio import RasterioIOError

items_s2 = pc.sign(search) # will perform serach and return ItemCollection that is signed. GeoJSON FeatureCollection whose features are all STAC Items

metadata = items_s2.to_dict()['features'][0]["properties"]
epsg = metadata["proj:epsg"]

gdal_session = stackstac.DEFAULT_GDAL_ENV.updated(always=dict(session=rasterio.session.AWSSession(aws_unsigned = True, endpoint_url = None)))
bands = ["AOT", "B01", "B02", "B03", "B04", "B05", "B06", "B07", "B08", "B8A", "B09", "B11", "B12", "WVP"]
stack = stackstac.stack(items_s2, epsg = epsg, assets = bands, dtype = "float32", properties = ["s2:product_uri", "s2:mean_solar_zenith", "s2:mean_solar_azimuth"], band_coords = False, bounds_latlon = padded_bbox_query, xy_coords = 'center', chunksize = 2048,errors_as_nodata=(RasterioIOError('.*'), ), gdal_env=gdal_session)

In [ ]:
bands = stack.band.values
stack["band"] = [f"s2_{b}" for b in stack.band.values]

stack = stack.to_dataset("band")

In [ ]:
import tempfile
import urllib
import uuid
from xml.dom import minidom
from datetime import datetime
import numpy as np

def parse_MTD_TL(in_file):
    """
    Parses the MTD_TL.xml metadata file provided by ESA.This metadata
    XML is usually placed in the GRANULE subfolder of a ESA-derived
    S2 product and named 'MTD_TL.xml'.

    The 'MTD_TL.xml' is available for both processing levels (i.e.,
    L1C and L2A). The function is able to handle both processing
    sources and returns some entries available in L2A processing level,
    only, as None type objects.

    The function extracts the most important metadata from the XML and
    returns a dict with those extracted entries.

    :param in_file:
        filepath of the scene metadata xml 8MTD_TL.xml)
    :return metadata:
        dict with extracted metadata entries
    """
    # parse the xml file into a minidom object
    xmldoc = minidom.parse(in_file)

    # now, the values of some relevant tags can be extracted:
    metadata = dict()

    # get tile ID of L2A product and its corresponding L1C counterpart
    tile_id_xml = xmldoc.getElementsByTagName("TILE_ID")
    # adaption to older Sen2Cor version
    check_l1c = True
    if len(tile_id_xml) == 0:
        tile_id_xml = xmldoc.getElementsByTagName("TILE_ID_2A")
        check_l1c = False
    tile_id = tile_id_xml[0].firstChild.nodeValue
    scene_id = tile_id.split(".")[0]
    metadata["SCENE_ID"] = scene_id

    # check if the scene is L1C or L2A
    is_l1c = False
    if check_l1c:
        try:
            l1c_tile_id_xml = xmldoc.getElementsByTagName("L1C_TILE_ID")
            l1c_tile_id = l1c_tile_id_xml[0].firstChild.nodeValue
            l1c_tile_id = l1c_tile_id.split(".")[0]
            metadata["L1C_TILE_ID"] = l1c_tile_id
        except Exception:
            logger.info(f"{scene_id} is L1C processing level")
            is_l1c = True

    # sensing time (acquisition time)
    sensing_time_xml = xmldoc.getElementsByTagName("SENSING_TIME")
    sensing_time = sensing_time_xml[0].firstChild.nodeValue
    metadata["SENSING_TIME"] = sensing_time
    metadata["SENSING_DATE"] = datetime.strptime(
        sensing_time.split("T")[0], "%Y-%m-%d"
    ).date()

    # number of rows and columns for each resolution -> 10, 20, 60 meters
    nrows_xml = xmldoc.getElementsByTagName("NROWS")
    ncols_xml = xmldoc.getElementsByTagName("NCOLS")
    resolutions = ["_10m", "_20m", "_60m"]
    # order: 10, 20, 60 meters spatial resolution
    for ii in range(3):
        nrows = nrows_xml[ii].firstChild.nodeValue
        ncols = ncols_xml[ii].firstChild.nodeValue
        metadata["NROWS" + resolutions[ii]] = int(nrows)
        metadata["NCOLS" + resolutions[ii]] = int(ncols)

    # EPSG-code
    epsg_xml = xmldoc.getElementsByTagName("HORIZONTAL_CS_CODE")
    epsg = epsg_xml[0].firstChild.nodeValue
    metadata["EPSG"] = int(epsg.split(":")[1])

    # Upper Left Corner coordinates -> is the same for all three resolutions
    ulx_xml = xmldoc.getElementsByTagName("ULX")
    uly_xml = xmldoc.getElementsByTagName("ULY")
    ulx = ulx_xml[0].firstChild.nodeValue
    uly = uly_xml[0].firstChild.nodeValue
    metadata["ULX"] = float(ulx)
    metadata["ULY"] = float(uly)
    # endfor

    # extract the mean zenith and azimuth angles
    # the sun angles come first followed by the mean angles per band
    zenith_angles = xmldoc.getElementsByTagName("ZENITH_ANGLE")
    metadata["SUN_ZENITH_ANGLE"] = float(zenith_angles[0].firstChild.nodeValue)

    azimuth_angles = xmldoc.getElementsByTagName("AZIMUTH_ANGLE")
    metadata["SUN_AZIMUTH_ANGLE"] = float(azimuth_angles[0].firstChild.nodeValue)

    # get the mean zenith and azimuth angle over all bands
    sensor_zenith_angles = [float(x.firstChild.nodeValue) for x in zenith_angles[1::]]
    metadata["SENSOR_ZENITH_ANGLE"] = np.mean(np.asarray(sensor_zenith_angles))

    sensor_azimuth_angles = [float(x.firstChild.nodeValue) for x in azimuth_angles[1::]]
    metadata["SENSOR_AZIMUTH_ANGLE"] = np.mean(np.asarray(sensor_azimuth_angles))

    # extract scene relevant data about nodata values, cloud coverage, etc.
    cloudy_xml = xmldoc.getElementsByTagName("CLOUDY_PIXEL_PERCENTAGE")
    cloudy = cloudy_xml[0].firstChild.nodeValue
    metadata["CLOUDY_PIXEL_PERCENTAGE"] = float(cloudy)

    degraded_xml = xmldoc.getElementsByTagName("DEGRADED_MSI_DATA_PERCENTAGE")
    degraded = degraded_xml[0].firstChild.nodeValue
    metadata["DEGRADED_MSI_DATA_PERCENTAGE"] = float(degraded)

    # the other tags are available in L2A processing level, only
    if not is_l1c:
        nodata_xml = xmldoc.getElementsByTagName("NODATA_PIXEL_PERCENTAGE")
        nodata = nodata_xml[0].firstChild.nodeValue
        metadata["NODATA_PIXEL_PERCENTAGE"] = float(nodata)

        darkfeatures_xml = xmldoc.getElementsByTagName("DARK_FEATURES_PERCENTAGE")
        darkfeatures = darkfeatures_xml[0].firstChild.nodeValue
        metadata["DARK_FEATURES_PERCENTAGE"] = float(darkfeatures)

        cs_xml = xmldoc.getElementsByTagName("CLOUD_SHADOW_PERCENTAGE")
        cs = cs_xml[0].firstChild.nodeValue
        metadata["CLOUD_SHADOW_PERCENTAGE"] = float(cs)

        veg_xml = xmldoc.getElementsByTagName("VEGETATION_PERCENTAGE")
        veg = veg_xml[0].firstChild.nodeValue
        metadata["VEGETATION_PERCENTAGE"] = float(veg)

        noveg_xml = xmldoc.getElementsByTagName("NOT_VEGETATED_PERCENTAGE")
        noveg = noveg_xml[0].firstChild.nodeValue
        metadata["NOT_VEGETATED_PERCENTAGE"] = float(noveg)

        water_xml = xmldoc.getElementsByTagName("WATER_PERCENTAGE")
        water = water_xml[0].firstChild.nodeValue
        metadata["WATER_PERCENTAGE"] = float(water)

        unclass_xml = xmldoc.getElementsByTagName("UNCLASSIFIED_PERCENTAGE")
        unclass = unclass_xml[0].firstChild.nodeValue
        metadata["UNCLASSIFIED_PERCENTAGE"] = float(unclass)

        cproba_xml = xmldoc.getElementsByTagName("MEDIUM_PROBA_CLOUDS_PERCENTAGE")
        cproba = cproba_xml[0].firstChild.nodeValue
        metadata["MEDIUM_PROBA_CLOUDS_PERCENTAGE"] = float(cproba)

        hcproba_xml = xmldoc.getElementsByTagName("HIGH_PROBA_CLOUDS_PERCENTAGE")
        hcproba = hcproba_xml[0].firstChild.nodeValue
        metadata["HIGH_PROBA_CLOUDS_PERCENTAGE"] = float(hcproba)

        thcirrus_xml = xmldoc.getElementsByTagName("THIN_CIRRUS_PERCENTAGE")
        thcirrus = thcirrus_xml[0].firstChild.nodeValue
        metadata["THIN_CIRRUS_PERCENTAGE"] = float(thcirrus)

        snowice_xml = xmldoc.getElementsByTagName("SNOW_ICE_PERCENTAGE")
        snowice = snowice_xml[0].firstChild.nodeValue
        metadata["SNOW_ICE_PERCENTAGE"] = float(snowice)

    return metadata



def angles_from_mspc(url):
    """
    Extract viewing angles from MS Planetary Computer
    metadata XML (this is a work-around until STAC provides the angles
    directly)

    :param url:
        URL to the metadata XML file
    :returns:
        extracted angles as dictionary
    """
    response = urllib.request.urlopen(pc.sign_url(url)).read()
    temp_file = os.path.join(tempfile.gettempdir(), f'{uuid.uuid4()}.xml')
    with open(temp_file, 'wb') as dst:
        dst.write(response)
    metadata = parse_MTD_TL(in_file=temp_file)
    # get sensor zenith and azimuth angle
    sensor_angles = ['SENSOR_ZENITH_ANGLE', 'SENSOR_AZIMUTH_ANGLE', 'SUN_ZENITH_ANGLE', 'SUN_AZIMUTH_ANGLE']
    sensor_angle_dict = {
        k: v for k, v in metadata.items() if k in sensor_angles}
    return sensor_angle_dict

In [ ]:
# Faster method to add the sensor angles

items_dict = {item.id: item for item in items_s2}
ordered_items = [items_dict[itemid] for itemid in stack.id.values]
sensor_zenith = []
for item in ordered_items:
    item = item.to_dict()
    granule_metadata_href = item["assets"]["granule-metadata"]["href"].split('xml')[0] + 'xml'
    sensor_angles = angles_from_mspc(granule_metadata_href)
    sensor_zenith.append(sensor_angles["SENSOR_ZENITH_ANGLE"])

In [ ]:
# Add sensor angles using product uri

# Move the extra coordinates to variables, as the time variable will be change to date
stack = stack.reset_coords('s2:product_uri').rename({'s2:product_uri': 'product_uri'})
stack = stack.reset_coords('s2:mean_solar_zenith').rename({'s2:mean_solar_zenith': 'mean_solar_zenith'})
stack = stack.reset_coords('s2:mean_solar_azimuth').rename({'s2:mean_solar_azimuth': 'mean_solar_azimuth'})

In [ ]:
# View items -> they are whole tiles, at different dates


import geopandas as gpd
from shapely.geometry import Polygon

df = gpd.GeoDataFrame.from_features(items_s2.to_dict(), crs="epsg:4326")
df.head()

In [ ]:
import matplotlib.pyplot as plt
import contextily as ctx

fig, ax = plt.subplots(figsize=(8,8))
df.plot(ax=ax, edgecolor='red', facecolor='none')
ax.set_xlim((df.total_bounds[0]-0.5, df.total_bounds[2]+0.5))
ax.set_ylim((df.total_bounds[1]-0.1, df.total_bounds[3]+0.1))
ctx.add_basemap(ax, crs=df.crs)

## Trying download approach

In [ ]:
# Figuring out the tile geometry for tile 32TMT

import planetary_computer as pc 
import stackstac
import rasterio
from rasterio import RasterioIOError
import geopandas as gpd
import os
import pystac_client
from pystac_client.stac_api_io import StacApiIO

# Setup client

URL = 'https://planetarycomputer.microsoft.com/api/stac/v1'

stac_api_io = StacApiIO()
stac_api_io.session.verify = os.environ.get('CURL_CA_BUNDLE')
catalog = pystac_client.Client.open(URL,stac_io=stac_api_io)

# Define what we search

lon_lat = (8.5417, 47.3796) # Zurich, in 32TMT
xy_shape = (128, 128)
resolution = 10
time_interval = "2021-07-01/2021-07-31"

bbox_query = bbox(lon_lat=lon_lat, xy_shape=xy_shape, resolution=resolution)
padded_bbox_query = padded_bbox(bbox_query, xy_shape)

search = catalog.search(
        bbox = padded_bbox_query,
        collections=["sentinel-2-l2a"],
        datetime=time_interval
) # retruns a ItermSearch object (search object of STAC API)


# Query data

items_s2 = pc.sign(search) # will perform serach and return ItemCollection that is signed. GeoJSON FeatureCollection whose features are all STAC Items

metadata = items_s2.to_dict()['features'][0]["properties"]
epsg = metadata["proj:epsg"]

gdal_session = stackstac.DEFAULT_GDAL_ENV.updated(always=dict(session=rasterio.session.AWSSession(aws_unsigned = True, endpoint_url = None)))
bands = ["AOT", "B01", "B02", "B03", "B04", "B05", "B06", "B07", "B08", "B8A", "B09", "B11", "B12", "WVP"]
stack = stackstac.stack(items_s2, epsg = epsg, assets = bands, dtype = "float32", properties = ["sentinel:product_id"], band_coords = False, bounds_latlon = padded_bbox_query, xy_coords = 'center', chunksize = 2048,errors_as_nodata=(RasterioIOError('.*'), ), gdal_env=gdal_session)


# View data
df = gpd.GeoDataFrame.from_features(items_s2.to_dict(), crs="epsg:4326")

print(df.geometry.values[0])

In [ ]:
# Query data

specs = {
    "lon_lat": (df.geometry.values[0].centroid.x, df.geometry.values[0].centroid.y), # center pixel (lon-lat)
    "xy_shape": (1024, 1024), # width, height of cutout around center pixel
    "resolution": 10, # in meters.. will use this on a local UTM grid..
    "time_interval": "2021-01-01/2021-01-31",
    "providers": [
        {
            "name": "s2",
            "kwargs": {"bands": ["AOT", "B01", "B02", "B03", "B04", "B05", "B06", "B07", "B08", "B8A", "B09", "B11", "B12", "WVP"], "best_orbit_filter": False, "five_daily_filter": False, "brdf_correction": True, "cloud_mask": False, "aws_bucket": "planetary_computer"}
        }
        ]
}

mc = emc.load_minicube(specs, compute = True)

In [ ]:
# Compare bbox that is queried and extent of data returned

from shapely.geometry import Polygon
import matplotlib.pyplot as plt
import contextily as ctx
import geopandas as gpd


bbox_query = bbox(lon_lat=specs["lon_lat"], xy_shape=specs["xy_shape"], resolution=specs["resolution"])
padded_bbox_query = padded_bbox(bbox_query, specs["xy_shape"])


# Create a Polygon from the coordinates
polygon_bbox = Polygon([
    (bbox_query[0], bbox_query[1]),
    (bbox_query[2], bbox_query[1]),
    (bbox_query[2], bbox_query[3]),
    (bbox_query[0], bbox_query[3]),
    (bbox_query[0], bbox_query[1])
])

polygon_padded_bbox = Polygon([
    (padded_bbox_query[0], padded_bbox_query[1]),
    (padded_bbox_query[2], padded_bbox_query[1]),
    (padded_bbox_query[2], padded_bbox_query[3]),
    (padded_bbox_query[0], padded_bbox_query[3]),
    (padded_bbox_query[0], padded_bbox_query[1])
])

# Create a GeoDataFrame with the Polygon
gdf_bbox = gpd.GeoDataFrame({'geometry': [polygon_bbox]}, crs='EPSG:4326')
gdf_padded_bbox = gpd.GeoDataFrame({'geometry': [polygon_padded_bbox]}, crs='EPSG:4326')

# Get extent of the data
point_ul = (mc['lon'].values[0], mc['lat'].values[0])
point_ur = (mc['lon'].values[-1], mc['lat'].values[0])

# Plot the GeoDataFrame
fig, ax = plt.subplots(figsize=(6, 6))
gdf_bbox.plot(ax=ax, alpha=0.5, edgecolor='blue', facecolor='none')
gdf_padded_bbox.plot(ax=ax, alpha=0.5, edgecolor='red', facecolor='none')
plt.plot(point_ul[0], point_ul[1], 'ko', markersize=3)  
plt.plot(point_ur[0], point_ur[1], 'ko', markersize=3)  


# Add contextily background
ctx.add_basemap(ax, crs=gdf_bbox.crs)

plt.show()